# Required Imports

In [1]:
import math
from loguru import logger as lg
import numpy as np
from time import perf_counter
from tqdm import tqdm
from copy import deepcopy
import random
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import collections
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import dendrogram as Dendrogram

In [2]:
from manim import *
from manim_editor import PresentationSectionType

Manim Community v0.13.1

In [3]:
import sklearn.datasets as data
from sklearn.cluster import DBSCAN
import hdbscan
from hdbscan._hdbscan_tree import compute_stability

# Slide 1

In [4]:
data_points = np.load(r"data\clusterable_data.npy")

In [5]:
def plot_clusters(data_points, algorithm, args, kwds, make_cluster):
    
    # Getting Lables
    if make_cluster:
        labels = algorithm(*args, **kwds).fit_predict(data_points)

    # Settings
    sns.set_context('poster')
    sns.set_style('white')
    sns.set_color_codes()
    plot_kwds = {'alpha' : 0.5, 's' : 10, 'linewidths':0}
    
    # Set Colors
    if make_cluster:
        palette = sns.color_palette('deep', np.unique(labels).max() + 1)
        colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    else:
        colors = [(0.0, 0.0, 0.0) for _ in range(len(data_points))]

    # Ploting
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot()
    ax.scatter(data_points.T[0], data_points.T[1], c=colors, **plot_kwds)

    # Print Eps value
    if make_cluster:
        plt.text(-0.5, 0.8, f'Eps: {kwds["eps"]:.3f}', fontsize=14)

    # Remove ticks
    plt.xticks([])
    plt.yticks([])
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)

    # Draw fig
    fig.canvas.draw()

    # Create buffer
    buf = fig.canvas.buffer_rgba()

    # Create Image Mobject
    img = ImageMobject(buf).scale(1)
    plt.close(fig)
    
    return img, fig

In [6]:
params = "-v WARNING    -ql --disable_caching --progress_bar leave Slide1"
paramsc = "-v WARNING    -ql --progress_bar leave Slide1"
paramsk = "-v WARNING --save_sections  -qk --disable_caching --progress_bar leave Slide1"

class Slide1(Scene):
    def construct(self):

        self.next_section("Title", type=PresentationSectionType.NORMAL)
        title = Title('HDBSCAN', match_underline_width_to_text=True, color=BLUE_D).move_to(ORIGIN).shift(UP).scale(2)
        sub_title = Text('Improving the DBSCAN Algorithm', color=RED_C).shift(DOWN*0.5).scale(.5)
        self.add(title, sub_title)
        self.wait()
        
        self.next_section("DBSCAN Intro", type=PresentationSectionType.NORMAL)
        self.play(FadeOut(title), FadeOut(sub_title))
        self.wait()
        title = Text("DBSCAN", color=WHITE).scale(1.5)
        title.to_edge(UP)
        self.play(Write(title))

        self.next_section("DBSCAN Full-Form", type=PresentationSectionType.NORMAL)
        subtitle = Text("Density-Based Spatial Clustering of Applications with Noise", font="Montserrat", color=WHITE).scale(0.5).next_to(title, DOWN)
        self.add(subtitle)
        self.wait()

        info = Text("by Martin Ester, Hans-Peter Kriegel, Jörg Sander, Xiaowei Xu, 1996", font="Montserrat", color=WHITE).scale(0.5).next_to(title, DOWN).shift(DOWN*0.5)
        self.add(info)
        self.wait()
        
        self.next_section("DBSCAN - Example Intro", type=PresentationSectionType.NORMAL)
        self.play(FadeOut(subtitle), FadeOut(info))

        # PLot points
        img, _ = plot_clusters(data_points, DBSCAN, (), {'eps': 0.022}, False)
        img.scale(0.5).to_edge(DOWN)
        self.play(FadeIn(img))

        self.next_section("DBSCAN - Example", type=PresentationSectionType.NORMAL)
        # Plot points with clusters
        img2, _ = plot_clusters(data_points, DBSCAN, (), {'eps': 0.022}, True)
        img2.scale(0.5).to_edge(DOWN)
        self.play(Transform(img, img2, run_time=2))

        self.next_section("DBSCAN - Problems", type=PresentationSectionType.NORMAL)
        # Problem with DBSCAN
        title2 = Text("Problem with DBSCAN", color=WHITE)
        title2.scale(1.5).to_edge(UP)
        self.play(TransformMatchingShapes(title, title2))

        # Plot points with clusters
        img3, _ = plot_clusters(data_points, DBSCAN, (), {'eps': 0.001}, True)
        img3.scale(0.5).to_edge(DOWN)
        self.play(Transform(img, img3))

        eps_min = 0.001
        eps = 0.001
        eps_max = 0.03

        tr_eps = ValueTracker(eps)
        image, _ = plot_clusters(data_points, DBSCAN, (), {'eps':eps}, True)
        image.scale(0.5).to_edge(DOWN)

        self.play(Transform(img, image))

        def update_image(mob):
            new_mob, _ = plot_clusters(data_points, DBSCAN, (), {'eps':tr_eps.get_value()}, True)
            new_mob.scale(0.5).to_edge(DOWN)
            mob.become(new_mob)

        # image.add_updater(update_image)
        img.add_updater(update_image)

        self.next_section("DBSCAN - Epsilon Instability", type=PresentationSectionType.COMPLETE_LOOP)
        self.play(tr_eps.animate.set_value(eps_max), run_time=10, rate_func=rate_functions.linear)
        self.play(tr_eps.animate.set_value(eps_min), run_time=10, rate_func=rate_functions.double_smooth)

        # Write Title for HDBSCAN
        # Title
        self.next_section("Transition to HDBSCAN", type=PresentationSectionType.NORMAL)
        # Fade out
        self.remove(img2, img3, image)
        self.play(FadeOut(title2), FadeOut(img))
        self.wait()
        title_Slide3 = Title("HDBSCAN", color=WHITE, match_underline_width_to_text=True).scale(1.5).to_edge(UP)
        self.play(Write(title_Slide3))
        self.wait()


In [7]:
%%manim $paramsk
plt.rcParams['figure.dpi'] = 300


Animation 3: Write(Text('DBSCAN')): 100%|##########| 60/60 [00:06<00:00,  8.83it/s]
Animation 6: FadeOut(Text('Density-Based Spatial Clustering of Applications with Noise')), etc.: 100%|##########| 60/60 [00:06<00:00,  9.62it/s]
Animation 13: _MethodAnimation(ValueTracker): 100%|##########| 600/600 [04:51<00:00,  2.06it/s]
Animation 14: FadeOut(Text('Problem with DBSCAN')), etc.: 100%|##########| 60/60 [00:57<00:00,  1.05it/s]
Animation 16: Write(Title('HDBSCAN')): 100%|##########| 60/60 [00:07<00:00,  8.17it/s]


# Slides 3

In [8]:
name = 'HDBSCAN_Presentation'
params = f"-v WARNING -ql --disable_caching --progress_bar leave {name}"
paramsc = f"-v WARNING -ql  --progress_bar leave {name}"
paramsk = f"-v WARNING --save_sections --disable_caching -qk --progress_bar leave {name}"
# paramsk = f"-v WARNING -qk --progress_bar leave {name}"
paramssk = f"-v WARNING -s -qk --disable_caching --progress_bar leave {name}"
paramssl = f"-v WARNING -s -ql --progress_bar leave {name}"


class HDBSCAN_Presentation(ZoomedScene):

    # Generate the Data required for explaination example
    def generate_example_data(self, k=5, data_path=None):

        print('Started Data Generation')
        start_time = perf_counter()

        # Import the data
        if data_path:
            self.data_points = np.load(r"data\clusterable_data.npy")
            self.grp_ids          = [(0, 400), (400, 600), (600, 800), (800, 1200), (1200, 1600), (1600, 1900), (1900, 2309)]
        else:
            self.data_points = data.make_blobs(n_samples=50, centers=2, cluster_std=0.1, n_features=2, random_state=42, center_box=(0, 1))[0]

        # Perform the HDBSCAN on the data
        self.k         = k
        self.clusterer = hdbscan.HDBSCAN(min_cluster_size=self.k, gen_min_span_tree=True)
        self.clusterer.fit(self.data_points)

        # Get the required data from the clusterer
        self.labels              = self.clusterer.labels_
        self.probabilities       = self.clusterer.probabilities_
        self.min_spanning_tree   = self.clusterer.minimum_spanning_tree_
        self.single_linkage_tree = self.clusterer.single_linkage_tree_
        self.condensed_tree      = self.clusterer.condensed_tree_

        # Data for Step 1: Computing the Core points
        self.distance_matrix = hdbscan.validity.pairwise_distances(self.data_points, metric='euclidean')
        self.core_radii      = []
        self.core_circles    = []
        self.sorted_dots     = []

        # Calculations for Dendrogram
        self.linkage     = self.single_linkage_tree.to_numpy()
        self.dend_dict   = Dendrogram(self.linkage, get_leaves=True, no_plot=True)
        self.icoord      = self.dend_dict['icoord']
        self.dcoord      = self.dend_dict['dcoord']
        self.ivl         = self.dend_dict['ivl']
        self.heights     = self.linkage[:,2]
        self.max_height  = max(self.heights)
        self.min_height  = min(self.heights)

        # Calculations for Condensed Tree
        plot_data            = self.condensed_tree.get_plot_data()
        self.bar_centers     = plot_data['bar_centers']
        self.bar_tops        = plot_data['bar_tops']
        self.bar_bottoms     = plot_data['bar_bottoms']
        self.bar_widths      = plot_data['bar_widths']
        self.line_xs         = plot_data['line_xs']
        self.line_ys         = plot_data['line_ys']
        self.cluster_bounds  = plot_data['cluster_bounds']

        print('Data Generation Completed')
        print(f'Time Taken: {perf_counter() - start_time}\n\n')

    # Creating starting title and algorithm description
    def create_title_page(self):
        # Title
        self.title_Slide3 = Title("HDBSCAN", color=WHITE, match_underline_width_to_text=True).scale(1.5).to_edge(UP)

        # Print the Algorithm
        algorithm = Text("Algorithm", color=RED).scale(.6).next_to(self.title_Slide3, DOWN*3)
        l1 = Line(LEFT*3, RIGHT*4, color=RED)
        l2 = Line(LEFT*3, RIGHT*4, color=RED)
        self.title_grp = VGroup(l1, algorithm, l2).arrange(DOWN, buff=0.1)
        algorithm.shift(LEFT)
        l1.shift(LEFT)
        l2.shift(LEFT)

        # Steps
        step1 = Tex(r'$1.$ Compute the core distance.', color=BLUE_D).scale(0.55).next_to(self.title_grp, DOWN)
        step2 = Tex(r'$2.$ Compute an Mutual Reachability Distance (MRD).', color=BLUE_D).scale(0.55)
        step3 = Tex(r'$3.$ Build a Minimum Spanning Tree, using MRD', color=BLUE_D).scale(0.55)
        step4 = Tex(r'$4.$ Construct Cluster Hierarchy using Single Linkage', color=BLUE_D).scale(0.55)
        step5 = Tex(r'$5.$ Condense the clusters using $k$', color=BLUE_D).scale(0.55)
        step6 = Tex(r'$6.$ Extract clusters', color=BLUE_D).scale(0.55)

        self.steps = VGroup(step1, step2, step3, step4, step5, step6).arrange(DOWN, center=False, aligned_edge=LEFT, buff=0.3)

        self.algorithm_group = VGroup(self.title_grp, self.steps).shift(UP)
        self.steps.shift(LEFT*1.5)
        self.algorithm_group.shift(RIGHT)

        # Playing the Scene
        self.add(self.title_Slide3)
        self.play(Write(self.title_grp))
        self.wait()
        self.play(AnimationGroup(
            *[FadeIn(step, shift=UP) for step in self.steps],
            lag_ratio=1
        ))

    # Data Creating Animation
    def create_explaination_data(self):
        self.dots = VGroup()
        p_scale = 8
        for  idx, (x, y) in enumerate(self.data_points):
            dot = Dot(point=[x*p_scale, y*p_scale, 0], color=WHITE).scale(0.5)  # type: ignore
            self.dots.add(dot)

        self.dots.move_to(ORIGIN)
        background = ScreenRectangle(aspect_ratio=1)
        background.set(height=6.5)
        background.set_fill(BLACK, 1)
        background.set_stroke(width=0)
        background.move_to(ORIGIN)
        self.dots.add_to_back(background)
        self.dots.shift(RIGHT*3.5)

        # The Animation
        self.play(self.algorithm_group.animate.to_edge(LEFT).scale(0.8).shift(UP*1.3+LEFT), self.title_Slide3.animate.to_corner(UL))
        self.wait()
        self.add(self.dots.submobjects[0])
        self.play(AnimationGroup(
            *[GrowFromCenter(dot) for dot in self.dots.submobjects[1:]],
            lag_ratio=0.05,
            rate_func=rate_functions.ease_in_cubic
        ))

    # Focus on step
    def focus_on_step(self, step_idx):
        step = self.steps[step_idx]
        return step.animate.set_color(YELLOW).scale(1.2).shift(RIGHT*.3)
    
    # Remove focus on step
    def remove_focus_on_step(self, step_idx):
        step = self.steps[step_idx]
        return step.animate.set_color(BLUE_D).scale(1/1.2).shift(LEFT*.3)

    # Show Step-By-Step Core Distance Animation
    def show_core_distance(self, dot_idx, frame):  

        # Get distances from distance matrix for given dot
        dists = self.distance_matrix[dot_idx-1]

        # Get k-Nearst Neighbors
        pts = []
        while len(pts) <= self.k + 1:
            pt_idx = np.argmin(dists)
            pts.append(self.dots[pt_idx+1])
            dists[pt_idx] = np.inf
        
        old_radius = euclidean(self.dots[dot_idx].get_center(), pts[1].get_center())
        self.core_circles.append(Circle(radius=old_radius, color=BLUE, fill_opacity=0.2, stroke_width=0.5).move_to(self.dots[dot_idx].get_center()))

        frame_h = frame.height
        if 2*old_radius > frame_h:
            frame_s = (2 * old_radius) / (frame_h)
        else:
            frame_s = 1

        self.play(self.dots[dot_idx].animate.set_color(RED))
        self.play(frame.animate.scale(frame_s), Create(self.core_circles[-1]))
        self.wait(0.1)
        
        for pt in pts[2:]:
            pt = pt.get_center()
            new_radius = euclidean(self.dots[dot_idx].get_center(), pt)
            s = new_radius / old_radius

            frame_h = frame.height
            if 2*new_radius > frame_h:
                frame_s = (2 * new_radius) / (frame_h)
            else:
                frame_s = 1

            self.play(frame.animate.scale(frame_s), self.core_circles[-1].animate.scale(s))
            old_radius = new_radius
            self.wait(0.1)
        
        self.wait()
        self.play(self.dots[dot_idx].animate.set_color(WHITE))
        self.core_radii.append(old_radius)
        self.sorted_dots.append(self.dots[dot_idx])

    # Get the core distance for given dot
    def get_core_circle(self, dot_idx):
        dists = self.distance_matrix[dot_idx-1]
        pts = []
        while len(pts) <= (self.k + 1):
            pt_idx = np.argmin(dists)
            pts.append(self.dots[pt_idx+1])
            dists[pt_idx] = np.inf
        
        pt = pts[-1]

        radius = euclidean(self.dots[dot_idx].get_center(), pt.get_center())
        self.core_radii.append(radius)
        core_circle = Circle(radius=radius, color=BLUE, fill_opacity=0.2, stroke_width=0.5).move_to(self.dots[dot_idx].get_center())
        self.core_circles.append(core_circle)
        self.sorted_dots.append(self.dots[dot_idx])
        return core_circle

    ## Performing Step 1 - Finding Core Distances
    def Step1(self):

        # Setting up the Zoom Camera and Frame
        zoomed_camera = self.zoomed_camera
        frame = zoomed_camera.frame
        frame.set_color(PURPLE)

        # Setting up the Zoomed Display
        zoomed_display = self.zoomed_display
        zoomed_display_frame = zoomed_display.display_frame
        zoomed_display_frame.set_color(RED)

        zd_rect = BackgroundRectangle(zoomed_display, fill_opacity=0, buff=MED_SMALL_BUFF)
        self.add_foreground_mobject(zd_rect)

        unfold_camera = UpdateFromFunc(zd_rect, lambda rect: rect.replace(zoomed_display))
        zoomed_display.to_edge(DOWN).shift(LEFT*3)

        dot_idx = 28
        frame.move_to(self.dots[dot_idx].get_center())
        self.play(Create(frame))
        self.activate_zooming()
        self.play(self.get_zoomed_display_pop_out_animation(), unfold_camera)
        self.wait(0.5)
        self.play(self.dots[dot_idx].animate.set_color(RED))
        self.wait()

        # Show Step-By-Step Core Distance Animation
        self.show_core_distance(dot_idx, frame)

        dot_idx = 43
        self.play(frame.animate.move_to(self.dots[dot_idx].get_center()))
        frame_h = frame.height
        self.wait(0.5)

        # Show Step-By-Step Core Distance Animation
        self.show_core_distance(dot_idx, frame)

        # Remove Zoomed Display
        self.play(self.get_zoomed_display_pop_out_animation(), unfold_camera, rate_func=lambda t: smooth(1 - t))
        self.play(Uncreate(zoomed_display_frame), FadeOut(frame))
        self.wait()

        # Get Core Circles for remaining dots
        circles = []
        for dot_idx in range(1, len(self.dots)):
            
            if dot_idx == 28 or dot_idx == 43:
                continue
            circles.append(self.get_core_circle(dot_idx))
        
        c1 = interpolate_color(YELLOW, RED, min(1, ((self.core_radii[0] - np.min(self.core_radii)) / np.mean(self.core_radii)) * 0.75))
        self.play(self.core_circles[0].animate.set_color(c1))
        self.wait(0.5)

        c2 = interpolate_color(YELLOW, RED, min(1, ((self.core_radii[1] - np.min(self.core_radii)) / np.mean(self.core_radii)) * 0.75))
        self.play(self.core_circles[1].animate.set_color(c2))

        for idx in range(len(circles)):
            c = interpolate_color(YELLOW, RED, min(1, ((self.core_radii[idx] - np.min(self.core_radii)) / np.mean(self.core_radii)) * 0.75))
            circles[idx].set_color(c)

        self.play(
            AnimationGroup(
                *[Create(circle) for circle in circles],
                lag_ratio=0.1,
                rate_func=rate_functions.ease_in_out_quart
            )
        )
        self.wait()

        self.play(
            AnimationGroup(
                self.sorted_dots[0].animate.set_color(c1),
                self.sorted_dots[1].animate.set_color(c2),
                *[self.sorted_dots[idx].animate.set_color(self.core_circles[idx].color) for idx in range(2, len(self.sorted_dots))],
                *[FadeOut(self.core_circles[idx], scale=0.5) for idx in range(len(self.sorted_dots))],
            ) 
        )

    # Show Mutual Reachability Distance Animation for given dots
    @lg.catch
    def present_mrd(self, idx1, idx2):

        # Choose Points
        dot1 = self.dots[idx1]
        dot2 = self.dots[idx2]
        old_color1 = dot1.get_color()
        old_color2 = dot2.get_color()
        self.play(dot1.animate.set_color(self.c1_color), Flash(dot1), self.mrd.animate.set_color_by_tex('p_i', self.c1_color))
        self.wait()

        self.play(dot2.animate.set_color(self.c2_color), Flash(dot2), self.mrd.animate.set_color_by_tex('p_j', self.c2_color))
        self.wait()

        def get_core_circle(dot_idx):
            dists = self.distance_matrix[dot_idx-1]
            pts = []
            while len(pts) <= (self.k + 1):
                pt_idx = np.argmin(dists)
                pts.append(self.dots[pt_idx+1])
                dists[pt_idx] = np.inf
            
            pt = pts[-1]

            radius = euclidean(self.dots[dot_idx].get_center(), pt.get_center())
            core_circle = Circle(radius=radius, color=BLUE, fill_opacity=0.2).move_to(self.dots[dot_idx].get_center())

            return core_circle

        core_circle1 = get_core_circle(idx1).set_color(self.c1_color)
        core_circle2 = get_core_circle(idx2).set_color(self.c2_color)

        self.play(Create(core_circle1), Indicate(self.mrd[5], color=self.c1_color, scale_factor=2))
        self.wait()
        self.play(Create(core_circle2), Indicate(self.mrd[9], color=self.c2_color, scale_factor=2))
        self.wait()

        # Draw line from dot1 to dot2
        line = Line(dot1.get_center(), dot2.get_center(), color=self.l_color)
        self.play(Create(line), Indicate(self.mrd[13], color=self.l_color, scale_factor=2))
        self.wait()

        len_dict = {
            core_circle1: core_circle1.width,
            core_circle2: core_circle2.width,
            line: line.get_length()
        }
        # sort according to length
        inds = [k for k, _ in sorted(len_dict.items(), key=lambda item: item[1], reverse=True)]

        # Show the distance
        self.play(Indicate(inds[0], color=inds[0].color, scale_factor=2), FadeOut(inds[1]), FadeOut(inds[2]))
        self.wait()

        self.play(FadeOut(inds[0]), dot1.animate.set_color(old_color1), dot2.animate.set_color(old_color2))
        self.wait()

    # Show relation between metric d and mrd
    @lg.catch
    def show_d_mrd_rel(self):

        # Select dots
        test_dots = VGroup()
        dot1 = self.dots[40]
        dot2 = self.dots[36]
        test_dots.add(dot1.copy())
        test_dots.add(dot2.copy())
        test_dots.shift(LEFT*2.5 + DOWN*0.5)
        self.play(TransformFromCopy(dot1, test_dots[0]), TransformFromCopy(dot2, test_dots[1]))
        self.wait()

        # Setting up the Zoom Camera and Frame
        zoomed_camera = self.zoomed_camera
        frame = zoomed_camera.frame
        frame.set_color(PURPLE)

        # Setting up the Zoomed Display
        zoomed_display = self.zoomed_display
        zoomed_display_frame = zoomed_display.display_frame
        zoomed_display_frame.set_color(RED)

        zd_rect = BackgroundRectangle(zoomed_display, fill_opacity=0, buff=MED_SMALL_BUFF)
        self.add_foreground_mobject(zd_rect)

        unfold_camera = UpdateFromFunc(zd_rect, lambda rect: rect.replace(zoomed_display))
        zoomed_display.to_edge(DR)

        frame.move_to(test_dots.get_center())
        frame.width=test_dots.width*1.5
        frame.height=test_dots.height*1.5
        self.play(Create(frame))
        self.activate_zooming()
        self.play(self.get_zoomed_display_pop_out_animation(), unfold_camera)
        self.wait(0.5)

        old_color1 = dot1.get_color()
        old_color2 = dot2.get_color()
        self.play(dot1.animate.set_color(self.c1_color), Flash(dot1), self.mrd.animate.set_color_by_tex('p_i', self.c1_color), test_dots[0].animate.set_color(self.c1_color))

        self.play(dot2.animate.set_color(self.c2_color), Flash(dot2), self.mrd.animate.set_color_by_tex('p_j', self.c2_color), test_dots[1].animate.set_color(self.c2_color))
        self.wait()

        line = Line(test_dots[0].get_center(), test_dots[1].get_center(), color=self.l_color, stroke_width=0.8)
        line_core = Line(dot1.get_center(), dot2.get_center(), color=self.l_color)
        self.play(Create(line_core), Create(line), Indicate(self.mrd[13], color=self.l_color, scale_factor=2))

        def get_core_circle(dot_idx):
            dists = self.distance_matrix[dot_idx-1]
            pts = []
            while len(pts) <= (self.k + 1):
                pt_idx = np.argmin(dists)
                pts.append(self.dots[pt_idx+1])
                dists[pt_idx] = np.inf
            
            pt = pts[-1]

            radius = euclidean(self.dots[dot_idx].get_center(), pt.get_center())
            core_circle = Circle(radius=radius, color=BLUE, fill_opacity=0.2).move_to(self.dots[dot_idx].get_center())

            return core_circle
        
        core_circle1 = get_core_circle(40).set_color(self.c1_color)
        core_circle2 = get_core_circle(36).set_color(self.c2_color)

        self.play(Create(core_circle1), Indicate(self.mrd[5], color=self.c1_color, scale_factor=2))
        self.play(Create(core_circle2), Indicate(self.mrd[9], color=self.c2_color, scale_factor=2))
        self.wait()

        len_dict = {
            core_circle1: core_circle1.width,
            core_circle2: core_circle2.width,
            line_core: line_core.get_length()
        }

        # sort according to length
        inds = [k for k, _ in sorted(len_dict.items(), key=lambda item: item[1], reverse=True)]

        test_dots[0].add_updater(lambda d: d.move_to(line.get_start()))  # type: ignore
        test_dots[1].add_updater(lambda d: d.move_to(line.get_end()))
        frame.add_updater(lambda f: f.set(width=test_dots.width*1.5))
        frame.add_updater(lambda f: f.set(height=test_dots.height*1.5))
        frame.add_updater(lambda f: f.set(center=test_dots.get_center()))
        self.wait()

        # Show the distance
        self.play(FadeOut(inds[1]), FadeOut(inds[2]))
        self.wait()
        self.play(line.animate.set_length(inds[0].width))
        self.wait()

        self.play(FadeOut(inds[0]), dot1.animate.set_color(old_color1), dot2.animate.set_color(old_color2))
        self.wait()

        self.play(self.get_zoomed_display_pop_out_animation(), unfold_camera, rate_func=lambda t: smooth(1 - t))
        self.play(Uncreate(zoomed_display_frame), FadeOut(frame), FadeOut(test_dots), FadeOut(zd_rect), FadeOut(line))
        self.wait()

    ## Performing Step 2 - Finding Mutual Reachability Distances
    @lg.catch
    def Step2(self):

        # The formula for mutual reachability distance is:
        self.mrd = MathTex(
                r'\text{mrd}(',
                r'p_i',
                r',\ \ ',
                r'p_j',
                r') =  \max\Big(',
                r'\text{core}_d',  # 5
                r'(',
                r'p_i',
                r'),\ \ ',
                r'\text{core}_d',  # 9
                r'(',
                r'p_j',
                r'),\ \ ',
                r'd',  # 13
                r'(',
                r'p_i',
                r',\ \ ',
                r'p_j',
                r')\Big)'
            ).scale(0.7).next_to(self.steps, DOWN*3).shift(RIGHT*1.5)

        self.play(Write(self.mrd))
        self.wait()

        self.c1_color = LIGHT_BROWN
        self.c2_color = BLUE_D
        self.l_color  = GREEN_D

        self.distance_matrix = hdbscan.validity.pairwise_distances(self.data_points, metric='euclidean')

        # d as mrd
        self.present_mrd(28, 37)

        # core_dist as mrd
        self.present_mrd(43, 13)

        # Relation between core_dist and metric d
        self.show_d_mrd_rel()

        # Indicate mrd for all points
        rect = SurroundingRectangle(self.mrd, color=BLACK, buff=0.1)
        self.play(AnimationGroup(
            *[Flash(dot) for dot in self.dots],
            lag_ratio=0.05
        ), ShowPassingFlash(rect.set_color(GREEN_D), run_time=3, time_width=2))
        self.wait(2)

        self.play(FadeOut(self.mrd))
        self.wait()

    ## Performing Step 3 - Creating the MST
    def Step3(self):

        min_spanning_array = self.min_spanning_tree.to_numpy()[:,:2]
        dist_array = self.min_spanning_tree.to_numpy()[:,2]

        # Create minimum spanning tree
        self.mst = []
        max_len  = max(dist_array)
        mean_len = np.mean(dist_array)
        min_len  = min(dist_array)
        for idx, pt in enumerate(min_spanning_array):
            c = interpolate_color(YELLOW, RED, min(dist_array[idx]/max_len, 1))
            line = Line(self.dots[int(pt[0])+1].get_center(), self.dots[int(pt[1])+1].get_center(), color=c)
            self.mst.append(line)
        
        self.play(AnimationGroup(*[Create(line) for line in self.mst], lag_ratio=0.5))

    ## Performing Step 4 - Build Single Linkage Hierarchy
    def Step4(self):
        lines           = []
        self.dendrogram = VGroup()
        m               = 10
        n               = 60

        label_position_x = set()
        label_position_y = set()
        dendrogram_dots  = []

        for idx in range(len(self.ivl)-1):

            x1, x2, x3, x4 = self.icoord[idx]
            y1, y2, y3, y4 = self.dcoord[idx]
            lable          = self.ivl[idx]

            x1, x2, x3, x4 = x1/n, x2/n, x3/n, x4/n
            y1, y2, y3, y4 = y1*m, y2*m, y3*m, y4*m

            c = WHITE

            line1 = Line([x1, y1, 0], [x2, y2, 0], color=c, stroke_width=0.75)
            line2 = Line([x2, y2, 0], [x3, y3, 0], color=c, stroke_width=0.75)
            line3 = Line([x3, y3, 0], [x4, y4, 0], color=c, stroke_width=0.75)
            l1_idx = len(lines) - 3
            l2_idx = len(lines) - 2
            l3_idx = len(lines) - 1
            lines.extend([line1, line2, line3])

            self.dendrogram.add(line1, line2)
            if y1 == 0:
                label_position_x.add(x1)
                label_position_y.add(y1)
                dendrogram_dots.append(Dot(point=[x1, y1, 0], color=RED).scale(0.5))
                self.dendrogram.add(dendrogram_dots[-1])

            self.dendrogram.add(line3)
            if y4 == 0:
                label_position_x.add(x4)
                label_position_y.add(y4)
                dendrogram_dots.append(Dot(point=[x4, y4, 0], color=RED).scale(0.5))
                self.dendrogram.add(dendrogram_dots[-1])

        self.dendrogram.scale(1.3).move_to(ORIGIN).shift(DOWN*2.5 + LEFT*1.5)

        dots_dict = {}
        for idx, dot in enumerate(dendrogram_dots):
            dots_dict[dot] = dot.get_center()

        sorted_dots_dict = {k: v for k, v in sorted(dots_dict.items(), key=lambda item: item[1][0])}
        sorted_dendrogram_dots = {}
        for idx, dot in enumerate(sorted_dots_dict.keys()):
            sorted_dendrogram_dots[dot] = self.dend_dict['leaves'][idx]

        self.play(
            AnimationGroup(*[Create(obj) if not isinstance(obj, Dot) else TransformFromCopy(self.dots[sorted_dendrogram_dots[obj] + 1], obj)for obj in self.dendrogram], lag_ratio=0.075)
        )

    # Get the bar for dendrogram for given dot
    def get_bar(self, idx, scale_m, max_width, min_width):
        width = self.bar_widths[idx] / scale_m
        height = (self.bar_tops[idx] - self.bar_bottoms[idx]) / (scale_m + 40)

        center_id = self.bar_centers[idx]
        center_x  = - center_id / scale_m
        center_y  = height

        return Rectangle(
            height = height,
            width  = width,
            color  = interpolate_color(YELLOW_D, RED, min((width - min_width) / (max_width - min_width), 1)),
            fill_opacity = 1,
        ), center_id, center_x, center_y

    ## Performing Step 5 - Create a Condenstion Tree
    def Step5(self):
        
        self.bars = VGroup()
        self.bar_clusters = {}

        scale_m = 25

        max_width = max(self.bar_widths[:-1]) / scale_m
        min_width = min(self.bar_widths) / scale_m
        
        for idx in range(len(self.bar_centers))[:-1]:

            bar, center_id, center_x, center_y = self.get_bar(idx, scale_m, max_width, min_width)

            if center_id not in self.bar_clusters:
                self.bar_clusters[center_id] = VGroup()
            bar_group = self.bar_clusters[self.bar_centers[idx]]

            if len(bar_group) == 0:
                bar.move_to([center_x, center_y, 0])
                bar_group.add(bar)

            else:
                bar.next_to(bar_group[-1], DOWN * bar_group[-1].height/2, buff=0)
                bar_group.add(bar)
            
            self.bars.add(bar)
        
        # Top bar
        bar, center_id, center_x, center_y = self.get_bar(-1, scale_m, max_width, min_width)
        bar.move_to([center_x, (center_y - bar.height)/2, 0])
        self.bars.add(bar)

        self.bars.move_to(self.dendrogram.get_center()).scale(1)

        x1_pt = self.dendrogram.get_boundary_point(UL)[0] - 0.5
        x2_pt = self.dendrogram.get_boundary_point(UR)[0] + 0.5
        y_pt  = self.bars.get_edge_center(UP)[1]
        revel_line = Line([x1_pt, y_pt, 0], [x2_pt, y_pt, 0], stroke_width=2).move_to(self.bars.get_edge_center(UP))

        def condensed_graph_updater(obj):
            
            if obj.get_center()[1] < revel_line.get_center()[1]:
                obj.set_opacity(0)
            else:
                obj.set_opacity(1)
        
        def dendrogram_updater(obj):

            if isinstance(obj, Dot):
                idx  = list(self.dendrogram).index(obj)
                line = self.dendrogram[idx + 1]
                y1   = line.get_start()[1]
                y2   = line.get_end()[1]
                y    = max(y1, y2)

            else:
                y1 = obj.get_start()[1]
                y2 = obj.get_end()[1]
                y  = max(y1, y2)
            
            if y > revel_line.get_center()[1]:
                obj.set_opacity(0)
            else:
                obj.set_opacity(1)

        for obj in self.dendrogram:
            obj.add_updater(dendrogram_updater)

        for obj in self.bars:
            obj.set_opacity(0)
            obj.add_updater(condensed_graph_updater)            

        self.add(self.bars)

        self.wait()
        self.play(Create(revel_line))

        self.wait(2)

        self.play(revel_line.animate.move_to(np.array(self.bars.get_edge_center(DOWN)) + np.array([0, 0.1, 0])), run_time=5)

        self.wait()
        self.play(Uncreate(revel_line))

    ## Performing Step 6 - Make Clusters
    def Step6(self):
        rects = []
        for idx in self.bar_clusters:
            grp = self.bar_clusters[idx]

            if len(list(grp)) > 1:
                rect = SurroundingRectangle(grp, buff=0.1)
                self.play(Create(rect))
                rects.append(rect)
        
        self.wait()

        self.play(AnimationGroup(
            *[Uncreate(line) for line in self.mst],
            FadeOut(self.bars),
            *[Uncreate(rect) for rect in rects]
        ))
        self.wait()

        # Get labels
        color = [RED, YELLOW, LIGHT_GRAY]

        self.play(
            AnimationGroup(*[Flash(self.dots[idx1 + 1]) for idx1 in range(len(self.labels))], lag_ratio=0.075),
            AnimationGroup(*[self.dots[idx2].animate.set_color(color[self.labels[idx2-1]]) for idx2 in range(1, len(self.dots))], lag_ratio=0.075)
        )

    # Main Construct Functions
    @lg.catch
    def construct(self):

        # Generate the data
        self.generate_example_data()

        # Create the title page
        self.next_section("Title Page", type=PresentationSectionType.NORMAL)
        self.create_title_page()

        # Create the Explaination Data
        self.next_section("Adding Data", type=PresentationSectionType.NORMAL)
        self.create_explaination_data()

        # Performing Step 1 - Finding Core Distances
        self.next_section("Step 1 - Finding Core Distances", type=PresentationSectionType.NORMAL)
        self.play(self.focus_on_step(0))
        self.Step1()

        # Performing Step 2 - Finding Mutual Reachability Distances
        self.next_section("Step 2 - Finding Mutual Reachability Distances", type=PresentationSectionType.NORMAL)
        self.play(self.remove_focus_on_step(0), self.focus_on_step(1))
        self.Step2()
        

        # Performing Step 3 - Building the Minimum Spanning Tree
        self.next_section("Step 3 - Building the Minimum Spanning Tree", type=PresentationSectionType.NORMAL)
        self.play(self.remove_focus_on_step(1), self.focus_on_step(2))
        self.Step3()

        # Performing Step 4 - Build the Single Linkage Hierarchy
        self.next_section("Step 4 - Build the Single Linkage Hierarchy", type=PresentationSectionType.NORMAL)
        self.play(self.remove_focus_on_step(2), self.focus_on_step(3))
        self.Step4()

        # Performing Step 5 - Create a Condenstion Tree
        self.next_section("Step 5 - Create a Condenstion Tree", type=PresentationSectionType.NORMAL)
        self.play(self.remove_focus_on_step(3), self.focus_on_step(4))
        self.Step5()

        # Performing Step 6 - Make Clusters
        self.next_section("Step 6 - Make Clusters", type=PresentationSectionType.NORMAL)
        self.play(self.remove_focus_on_step(4), self.focus_on_step(5))
        self.Step6()
        
        # Fade out the scene
        self.next_section('Transition to Final Example', type=PresentationSectionType.NORMAL)
        self.play(FadeOut(self.dots), FadeOut(self.algorithm_group), FadeOut(self.title_Slide3))


In [9]:
%%time
%%manim $paramsk
plt.rcParams['figure.dpi'] = 300

Started Data Generation
Data Generation Completed
Time Taken: 0.02777720001176931




Animation 0: Write(VGroup of 3 submobjects): 100%|##########| 60/60 [00:07<00:00,  7.55it/s]


# Final Test Example

In [ ]:
name = 'Final_Example'
params = f"-v WARNING -ql --disable_caching --progress_bar leave {name}"
paramsc = f"-v WARNING -ql --progress_bar leave {name}"
paramsk = f"-v WARNING --save_sections --disable_caching -qk --progress_bar leave {name}"
paramssk = f"-v WARNING -s -qk --disable_caching --progress_bar leave {name}"
paramssl = f"-v WARNING -s -ql --progress_bar leave {name}"

class Final_Example(ZoomedScene):

    # Generate required data
    def generate_data(self):

        print('Started Data Generation')
        start_time = perf_counter()

        # Import the data
        self.data_points = np.load("data\clusterable_data.npy")
        self.grp_ids          = [(0, 400), (400, 600), (600, 800), (800, 1200), (1200, 1600), (1600, 1900), (1900, 2309)]

        # Perform the HDBSCAN on the data
        self.k         = 15
        self.clusterer = hdbscan.HDBSCAN(min_cluster_size=self.k, gen_min_span_tree=True)
        self.clusterer.fit(self.data_points)

        # Get the required data from the clusterer
        self.labels              = self.clusterer.labels_
        self.probabilities       = self.clusterer.probabilities_
        self.min_spanning_tree   = self.clusterer.minimum_spanning_tree_
        self.single_linkage_tree = self.clusterer.single_linkage_tree_
        self.condensed_tree      = self.clusterer.condensed_tree_

        # Data for Step 1: Computing the Core points
        self.distance_matrix = hdbscan.validity.pairwise_distances(self.data_points, metric='euclidean')
        self.core_radii      = []
        self.core_circles    = []
        self.sorted_dots     = []

        # Calculations for Dendrogram
        self.linkage     = self.single_linkage_tree.to_numpy()
        self.dend_dict   = Dendrogram(self.linkage, get_leaves=True, no_plot=True)
        self.icoord      = self.dend_dict['icoord']
        self.dcoord      = self.dend_dict['dcoord']
        self.ivl         = self.dend_dict['ivl']
        self.heights     = self.linkage[:,2]
        self.max_height  = max(self.heights)
        self.min_height  = min(self.heights)

        # Calculations for Condensed Tree
        plot_data            = self.condensed_tree.get_plot_data()
        self.bar_centers     = plot_data['bar_centers']
        self.bar_tops        = plot_data['bar_tops']
        self.bar_bottoms     = plot_data['bar_bottoms']
        self.bar_widths      = plot_data['bar_widths']
        self.line_xs         = plot_data['line_xs']
        self.line_ys         = plot_data['line_ys']
        self.cluster_bounds  = plot_data['cluster_bounds']

        print('Data Generation Completed')
        print(f'Time Taken: {perf_counter() - start_time}\n\n')

    # Creating Title and Algorithm
    def create_title_and_algorithm(self):

        # Steps
        step0 = Tex(r'Step $0:$ Import the data points', color=BLUE_D).scale(0.55).to_edge(UP)
        step1 = Tex(r'Step $1:$ Compute the core distance', color=BLUE_D).scale(0.55).to_edge(UP)
        step2 = Tex(r'Step $2:$ Compute an Mutual Reachability Distance (MRD)', color=BLUE_D).scale(0.55).to_edge(UP)
        step3 = Tex(r'Step $3:$ Build a Minimum Spanning Tree, using MRD', color=BLUE_D).scale(0.55).to_edge(UP)
        step4 = Tex(r'Step $4:$ Construct Cluster Hierarchy using Single Linkage', color=BLUE_D).scale(0.55).to_edge(UP)
        step5 = Tex(r'Step $5:$ Condense the clusters using $k$', color=BLUE_D).scale(0.55).to_edge(UP)
        step6 = Tex(r'Step $6:$ Extract clusters', color=BLUE_D).scale(0.55).to_edge(UP)

        self.steps = VGroup(step0, step1, step2, step3, step4, step5, step6)

    # Step 0: Input Data
    def input_dots(self):

        # VGroup to store all the dots
        self.dots = VGroup()

        # The scale value for the dots position
        x_scale = 12
        y_scale = 5

        # Position the dots and add them to the VGroup
        print('Generating Dots')
        start_time = perf_counter()
        for  idx, (x, y) in enumerate(self.data_points):
            dot = Dot(point=[x*x_scale, y*y_scale, 0], color=WHITE, radius=0.025).scale(0.5)
            self.dots.add(dot)
        print('Dots Generated')
        print(f'Time Taken: {perf_counter() - start_time}\n\n')
        
        # Move the sots to the center of the screen
        self.dots.move_to(ORIGIN)
        background = ScreenRectangle(aspect_ratio=1)
        background.set_fill(BLACK, 1)
        background.set_stroke(width=0)
        background.move_to(ORIGIN)
        self.dots.add_to_back(background)

        self.add(self.dots.submobjects[0])

        print('Shuffling the dots')
        start_time = perf_counter()
        np.random.seed(43)
        shuffled_ids = list(range(1, len(self.dots)))
        np.random.shuffle(shuffled_ids)
        print('Dots Shuffled')
        print(f'Time Taken: {perf_counter() - start_time}\n\n')

        print('Starting Dot Generation Animation')
        start_time = perf_counter()
        self.play(AnimationGroup(
            *[GrowFromCenter(self.dots[id]) for id in tqdm(shuffled_ids, desc='Generating Dots')],
            lag_ratio=0.005,
            rate_func=rate_functions.linear
        ))
        print('Dot Generation Animation Completed')
        print(f'Time Taken: {perf_counter() - start_time}\n\n')

    # Helping Function for Step 1:
    # Compute core circle for given point
    def get_core_circle(self, dot_idx):

        # Get the Distance of the given point from all other points
        # The id of given point in the distance matrix is 1 less than the id of the dot in the dots VGroup
        dists = self.distance_matrix[dot_idx - 1]
        pts = []

        # Find the kth nearest point to the given point
        while len(pts) <= self.k + 1:
            pt_idx = np.argmin(dists)
            pts.append(self.dots[pt_idx + 1])
            dists[pt_idx] = np.inf
        
        # The kth nearest point is the last point in the list
        pt = pts[-1]

        # Get the distance between the given point and the kth nearest point
        radius = euclidean(self.dots[dot_idx].get_center(), pt.get_center())
        core_circle = Circle(radius=radius, color=BLUE, fill_opacity=0.2, stroke_width=0.5).move_to(self.dots[dot_idx].get_center())

        self.core_radii.append(radius)
        self.core_circles.append(core_circle)
        self.sorted_dots.append(self.dots[dot_idx])
        return core_circle

    # Step 1: Compute the core distance
    def compute_core_distances(self, testing:bool = False):

            circles = []
            for dot_idx in tqdm(range(1, len(self.dots)), desc='Generating Core Circles'):
                circles.append(self.get_core_circle(dot_idx))

            for idx in tqdm(range(len(circles)), desc='Coloring the Core Circles'):
                c = interpolate_color(YELLOW, RED, min(1, ((self.core_radii[idx] - np.min(self.core_radii)) / np.mean(self.core_radii)) * 0.75))
                circles[idx].set_color(c)

            for start_id, end_id in self.grp_ids:
                print(f'\nGrouping {start_id} to {end_id}')

                if testing:
                    pass
                else:
                    self.play(
                        AnimationGroup(
                            *[Create(circle) for circle in tqdm(circles[start_id: end_id], desc=f'Core Circles[{start_id}:{end_id}]')],
                            lag_ratio=0.01,
                            rate_func=rate_functions.ease_in_out_quart
                        )
                    )

                if testing:
                    [self.sorted_dots[idx].set_color(self.core_circles[idx].color) for idx in tqdm(range(start_id, end_id), desc=f'Coloring Dots[{start_id}:{end_id}]')]
                else:
                    self.play(
                        AnimationGroup(
                            *[self.sorted_dots[idx].animate.set_color(self.core_circles[idx].color) for idx in tqdm(range(start_id, end_id), desc=f'Coloring Dots[{start_id}:{end_id}]')],
                            *[ShrinkToCenter(self.core_circles[idx], scale=0.5) for idx in tqdm(range(start_id, end_id), desc=f'Shrinking Core Circles[{start_id}:{end_id}]')]
                        ) 
                    )

    # Drawing Single Linkage Tree
    def draw_single_linkage_tree(self):
        min_spanning_array = self.min_spanning_tree.to_numpy()[:,:2]
        dist_array = self.min_spanning_tree.to_numpy()[:,2]

        # Create minimum spanning tree
        self.mst = []
        max_len  = max(dist_array)
        mean_len = np.mean(dist_array)
        min_len  = min(dist_array)
        for idx, pt in enumerate(tqdm(min_spanning_array, desc='Creating MST')):
            c = interpolate_color(YELLOW, RED, min(dist_array[idx] / max_len, 1))
            line = Line(self.dots[int(pt[0]) + 1].get_center(), self.dots[int(pt[1]) + 1].get_center(), color=c, stroke_width=0.75)
            self.mst.append(line)
        
        self.play(AnimationGroup(*[Create(line) for line in self.mst], lag_ratio=0.005))

    # Get the bar from id
    def get_bar(self, idx):
        width = self.bar_widths[idx]
        try:
            if self.bar_centers[idx] == self.bar_centers[idx + 1]:
                height = abs(self.bar_bottoms[idx+1] - self.bar_bottoms[idx])
            else:
                height = self.bar_tops[idx]
        except IndexError:
            height = self.bar_tops[idx]

        center_id = self.bar_centers[idx]
        center_x  = - center_id
        center_y  = - ((height / 2) + self.bar_bottoms[idx])

        bar = Rectangle(
            height = height,
            width  = width,
            color  = interpolate_color(YELLOW_D, RED, min((width - self.min_width) / (self.max_width - self.min_width), 1)),
            fill_opacity = 1,
        )

        return bar, center_id, center_x, center_y

    # Create Bars
    def create_bars(self):

        self.bars = VGroup()
        self.bar_clusters = {}
        self.condensed_tree_grp = VGroup()

        self.max_width = max(self.bar_widths[:-1])
        self.min_width = min(self.bar_widths)

        for idx in range(len(self.bar_centers)):

            bar, center_id, center_x, center_y = self.get_bar(idx)

            if center_id not in self.bar_clusters:
                self.bar_clusters[center_id] = VGroup()

            bar_group = self.bar_clusters[self.bar_centers[idx]]

            if len(bar_group) == 0:
                bar.move_to([center_x, center_y, 0])
                bar_group.add(bar)

            else:
                bar.next_to(bar_group[-1], DOWN * bar_group[-1].height/2, buff=0)
                bar_group.add(bar)
            
            self.bars.add(bar)

    # Create Condensed Cluster Outline
    def create_condensed_cluster_outline(self):

        self.condensed_cluster_outline = VGroup()
        self.outline_dict = {}

        for cluster_id in self.cluster_bounds:

            x_min, x_max, y_min,  y_max = self.cluster_bounds[cluster_id]
            width = abs(x_max - x_min) + 500
            height = abs(y_max - y_min)
            x = (x_max + x_min) / 2
            y = (y_max + y_min) / 2
            rect = Rectangle(height=height, width=width, color=WHITE, stroke_width=2).move_to([-x, -y, 0])

            self.condensed_cluster_outline.add(rect)
            self.outline_dict[cluster_id] = self.condensed_cluster_outline[-1]

    # Create Horizontal Lines
    def create_horizontal_lines(self):

        self.horizontal_lines = VGroup()

        for idx in range(len(self.line_xs)):

            x1, x2 = self.line_xs[idx]
            y1, y2 = self.line_ys[idx]
            line = Line(np.array([-x1, -y1, 0]), np.array([-x2, -y2, 0]), color=WHITE, stroke_width=2)
            self.horizontal_lines.add(line)

    # Create Cluster Numbers
    def create_cluster_numbers(self):
        self.cluster_nums = VGroup()
        for idx, cluster_id in enumerate(self.cluster_bounds):
            num  = Text(str(cluster_id), color=WHITE).scale(0.25).move_to(self.condensed_cluster_outline[idx].get_edge_center(UP))
            self.cluster_nums.add(num)

    # Compute Cluster Stability
    def compute_cluster_stability(self):
        self.cluster_stabilities = compute_stability(self.condensed_tree.to_numpy())
        self.braces = VGroup()
        for idx, cluster_id in enumerate(self.cluster_bounds):
            br = BraceLabel(self.condensed_cluster_outline[idx], f'{self.cluster_stabilities[cluster_id]:.2f}', brace_direction=RIGHT, color=WHITE, buff= 0.1, font_size=18)
            self.braces.add(br)
    
    # Create Cluster Hierarchy
    def create_cluster_hierarchy(self):
        self.cluster_hierarchy = {}
        for row in self.condensed_tree.to_numpy():
            parent, child, _, _ = row

            if child in self.cluster_bounds.keys():
                if parent in self.cluster_hierarchy.keys():
                    self.cluster_hierarchy[parent].append(child)
                else:
                    self.cluster_hierarchy[parent] = [child]

    # Perform cluster selection on given parent, children group
    def select_cluster(self, parent_id, child1_id, child2_id, show_steps:bool = False):

        scale_size = 0.5
        lambdas = VGroup()

        # Show step by step process
        if show_steps:

            # Highlight parent cluster
            self.play(Create(self.outline_dict[parent_id]))
            l_val = self.cluster_stabilities[parent_id]
            parent_lambda = Tex(f'{l_val:.2f}').scale(scale_size - 0.25).next_to(self.outline_dict[parent_id], RIGHT, buff=0.1)
            parent_lambda_copy = Tex(f'{l_val:.2f}').scale(scale_size).to_edge(UL)
            lambdas.add(parent_lambda_copy)
            self.play(Write(parent_lambda))

            # Highlight child clusters
            self.play(Create(self.outline_dict[child1_id]), Create(self.outline_dict[child2_id]))
            l1_val = self.cluster_stabilities[child1_id]
            l2_val = self.cluster_stabilities[child2_id]
            child1_lambdas = Tex(f'{l1_val:.2f}').scale(scale_size - 0.25).next_to(self.outline_dict[child1_id], RIGHT, buff=0.1)
            child2_lambdas = Tex(f'{l2_val:.2f}').scale(scale_size - 0.25).next_to(self.outline_dict[child2_id], RIGHT, buff=0.1)
            c_lambdas = VGroup(child1_lambdas, child2_lambdas)

            child_lambdas = Tex(f'{l1_val:.2f} + {l2_val:.2f}').scale(scale_size).next_to(parent_lambda_copy, RIGHT*2)
            lambdas.add(child_lambdas)
            self.play(Write(child1_lambdas), Write(child2_lambdas))
        
            # Compare lambda values
            self.play(TransformFromCopy(parent_lambda, parent_lambda_copy), TransformFromCopy(c_lambdas, child_lambdas))
            if l_val > l1_val + l2_val:
                sign = Tex(r'$\geq$').scale(scale_size).next_to(lambdas[0], RIGHT*0.65)
                lambdas.add(sign)
                self.play(Write(sign))
                self.play(FadeOut(lambdas[1]), FadeOut(lambdas[2]), FadeOut(c_lambdas), FadeOut(self.outline_dict[child1_id]), FadeOut(self.outline_dict[child2_id]))
                self.play(Indicate(self.outline_dict[parent_id], color=BLUE_D, scale_factor=1.5), self.outline_dict[parent_id].animate.set_color(BLUE_D), FadeOut(lambdas[0]))
                self.play(FadeOut(self.outline_dict[parent_id], run_time=.5))
                self.selected_lambdas.add(parent_lambda)
            else:
                sign = Tex(r'$\leq$').scale(scale_size).next_to(lambdas[0], RIGHT*0.65)
                lambdas.add(sign)
                self.play(Write(sign))
                self.play(FadeOut(lambdas[0]), FadeOut(lambdas[2]))
                self.cluster_stabilities[parent_id] = l1_val + l2_val
                changed_parent_lambda = Tex(f'{self.cluster_stabilities[parent_id]:.2f}').scale(scale_size - 0.25).next_to(self.outline_dict[parent_id], RIGHT, buff=0.1)
                self.play(TransformMatchingTex(parent_lambda, changed_parent_lambda), FadeOut(lambdas[1]), FadeOut(self.outline_dict[child1_id]), FadeOut(self.outline_dict[child2_id]), FadeOut(c_lambdas))
                self.play(FadeOut(changed_parent_lambda))
        else:
            
            # Highlight parent cluster
            l_val = self.cluster_stabilities[parent_id]
            parent_lambda = Tex(f'{l_val:.2f}').scale(scale_size - 0.25).next_to(self.outline_dict[parent_id], RIGHT, buff=0.1)

            # Highlight child clusters
            l1_val = self.cluster_stabilities[child1_id]
            l2_val = self.cluster_stabilities[child2_id]
            child1_lambdas = Tex(f'{l1_val:.2f}').scale(scale_size - 0.25).next_to(self.outline_dict[child1_id], RIGHT, buff=0.1)
            child2_lambdas = Tex(f'{l2_val:.2f}').scale(scale_size - 0.25).next_to(self.outline_dict[child2_id], RIGHT, buff=0.1)
            c_lambdas = VGroup(child1_lambdas, child2_lambdas)

            self.play(Create(self.outline_dict[parent_id]), Create(self.outline_dict[child1_id]), Create(self.outline_dict[child2_id]), Write(child1_lambdas), Write(child2_lambdas), Write(parent_lambda))
        
            # Compare lambda values
            if l_val > l1_val + l2_val:
                self.play(FadeOut(c_lambdas), Indicate(self.outline_dict[parent_id], color=BLUE_D, scale_factor=1.5), FadeOut(self.outline_dict[child1_id]), FadeOut(self.outline_dict[child2_id]))
                self.outline_dict[parent_id].set_color(BLUE_D)
                self.play(FadeOut(self.outline_dict[parent_id], run_time=.5))
                self.selected_lambdas.add(parent_lambda)
            else:
                self.cluster_stabilities[parent_id] = l1_val + l2_val
                changed_parent_lambda = Tex(f'{self.cluster_stabilities[parent_id]:.2f}').scale(scale_size - 0.25).next_to(self.outline_dict[parent_id], RIGHT, buff=0.1)
                self.play(TransformMatchingTex(parent_lambda, changed_parent_lambda), FadeOut(c_lambdas), FadeOut(self.outline_dict[child1_id]), FadeOut(self.outline_dict[child2_id]))
                self.play(FadeOut(changed_parent_lambda))

    # Perform Cluster Selection
    def perform_cluster_selection(self):

        # Create Cluster Hierarchy
        self.create_cluster_hierarchy()
        self.selected_lambdas = VGroup()

        # Show first parent with steps
        parent_id = list(self.cluster_hierarchy.keys())[-1]
        child1_id, child2_id = self.cluster_hierarchy[parent_id]
        self.select_cluster(parent_id, child1_id, child2_id, True)

        parent_id = list(self.cluster_hierarchy.keys())[-2]
        child1_id, child2_id = self.cluster_hierarchy[parent_id]
        self.select_cluster(parent_id, child1_id, child2_id, True)

        for parent_id in list(self.cluster_hierarchy.keys())[::-1][2:-1]:

            child1_id, child2_id = self.cluster_hierarchy[parent_id]
            self.select_cluster(parent_id, child1_id, child2_id)


    def create_condensed_tree(self):
        
        # Create Bars
        self.create_bars()
        self.condensed_tree_grp.add(self.bars)

        # Create Condensed Cluster Outlines
        self.create_condensed_cluster_outline()
        self.condensed_tree_grp.add(self.condensed_cluster_outline)

        # Create Horizontal Lines
        self.create_horizontal_lines()
        self.condensed_tree_grp.add(self.horizontal_lines)

        # Fit the Condensed Tree Group in Frame
        self.condensed_tree_grp.stretch_to_fit_height(5)
        self.condensed_tree_grp.stretch_to_fit_width(12)
        self.condensed_tree_grp.move_to(ORIGIN + DOWN)

        # Create Cluster Numbers
        self.create_cluster_numbers()
        self.condensed_tree_grp.add(self.cluster_nums)

        # Compute stability
        self.compute_cluster_stability()
        self.condensed_tree_grp.add(self.braces)

        # Add Background to Condensed Tree Group
        c = interpolate_color(WHITE, BLACK, 0.95)
        background = Rectangle(height=5.5, width=12.5, color=c, fill_opacity=1).move_to(ORIGIN+DOWN)#.shift(LEFT*0.5)
        background.set_stroke(width=1)
        self.condensed_tree_grp.add_to_back(background)
        self.play(FadeIn(background), run_time=1)

        self.play(
            AnimationGroup(*[DrawBorderThenFill(obj) for obj in self.bars], lag_ratio=0.025),
            AnimationGroup(*[Write(obj) for obj in self.horizontal_lines], lag_ratio=0.025)
        )
        self.wait()

        self.perform_cluster_selection()
        selected_clusters = self.condensed_tree._select_clusters()
        selected_outlines = VGroup()
        for idx, cluster_id in enumerate(self.cluster_bounds):
            if cluster_id in selected_clusters:
                self.condensed_cluster_outline[idx].set_color(BLUE_D)
                selected_outlines.add(self.condensed_cluster_outline[idx])
        
        self.play(AnimationGroup(*[Create(obj) for obj in selected_outlines]))

    # Extract the Cluster
    def extract_cluster(self):

        # Fadout the Condensed Tree
        self.play(FadeOut(self.selected_lambdas), FadeOut(self.bars), FadeOut(self.horizontal_lines), FadeOut(self.condensed_cluster_outline), FadeOut(self.condensed_tree_grp[0]))

        # Uncreate the Single Linkage Tree
        self.play(AnimationGroup(*[Uncreate(line) for line in self.mst]))

        # Get labels
        color = [RED_C, YELLOW_C, BLUE_D, TEAL_D, GREEN_C, PURPLE_D, LIGHT_GRAY]

        self.play(
            # AnimationGroup(*[Flash(self.dots[idx1 + 1]) for idx1 in tqdm(range(len(self.labels)))], lag_ratio=0.005),
            AnimationGroup(*[self.dots[idx2].animate.set_color(color[self.labels[idx2-1]]) for idx2 in tqdm(range(1, len(self.dots)))], lag_ratio=0.005)
        )
        print('Done')

    # Main Construct Function
    @lg.catch
    def construct(self):

        # Generate the data
        self.generate_data()
        
        # Create the title and algorithm
        self.create_title_and_algorithm()

        # Start with step 0 (Input Data)
        self.play(Write(self.steps[0]))
        self.wait()
        self.input_dots()
        self.wait()

        # Step 1: Compute the core distance
        self.next_section("Step 1: Compute the core distance", type=PresentationSectionType.NORMAL)
        self.play(TransformMatchingShapes(self.steps[0], self.steps[1]))
        self.wait()
        self.compute_core_distances()
        self.wait()

        # Step 2: Compute the Mutual Reachability Distance
        self.next_section("Step 2: Compute the Mutual Reachability Distance", type=PresentationSectionType.NORMAL)
        self.play(TransformMatchingShapes(self.steps[1], self.steps[2]))
        self.wait()

        # Step 3: Build a Minimum Spanning Tree
        self.next_section("Step 3: Build a Minimum Spanning Tree", type=PresentationSectionType.NORMAL)
        self.play(TransformMatchingShapes(self.steps[2], self.steps[3]))
        self.wait()
        self.draw_single_linkage_tree()
        self.wait()
        
        # Step 4: Compute Single Linkage Tree using MRD
        self.next_section("Step 4: Compute Single Linkage Tree using MRD", type=PresentationSectionType.NORMAL)
        self.play(TransformMatchingShapes(self.steps[3], self.steps[4]))
        self.wait()

        # Step 5: Condense the Single Linkage Tree
        self.next_section("Step 5: Condense the Single Linkage Tree", type=PresentationSectionType.NORMAL)
        self.play(TransformMatchingShapes(self.steps[4], self.steps[5]))
        self.wait()
        self.create_condensed_tree()
        self.wait()

        # Step 6: Extract the clusters
        self.next_section("Step 6: Extract the clusters", type=PresentationSectionType.NORMAL)
        self.play(TransformMatchingShapes(self.steps[5], self.steps[6]))
        self.wait()
        self.extract_cluster()
        self.wait(2)


In [ ]:
%%time
%%manim $paramsk
plt.rcParams['figure.dpi'] = 300

Started Data Generation
Data Generation Completed
Time Taken: 0.2316288000001805




Animation 0: Write(Tex('Step $0:$ Import the data points')): 100%|##########| 120/120 [00:10<00:00, 11.44it/s]


Generating Dots
Dots Generated
Time Taken: 1.2770554000017


Shuffling the dots
Dots Shuffled
Time Taken: 0.000149200001033023


Starting Dot Generation Animation


Animation 2: AnimationGroup(Group): 100%|##########| 753/753 [06:28<00:00,  1.94it/s]


Dot Generation Animation Completed
Time Taken: 407.70370980000007




...

Animation 17: AnimationGroup(Group): 100%|##########| 60/60 [00:17<00:00,  3.42it/s]



Grouping 1900 to 2309


Animation 37: Write(Tex('3.68')), etc.: 100%|##########| 60/60 [00:03<00:00, 15.20it/s]
Animation 38: TransformFromCopy(Tex('2536.68')), etc.: 100%|##########| 60/60 [00:04<00:00, 12.46it/s]
Animation 46: Write(Tex('1208.97')), etc.: 100%|##########| 60/60 [00:04<00:00, 14.07it/s]
Animation 47: TransformFromCopy(Tex('120.89')), etc.: 100%|##########| 60/60 [00:05<00:00, 10.06it/s]
Animation 52: Create(Rectangle), etc.: 100%|##########| 60/60 [00:04<00:00, 14.84it/s]
Animation 53: FadeOut(VGroup of 2 submobjects), etc.: 100%|##########| 60/60 [00:03<00:00, 15.75it/s]
Animation 58: Create(Rectangle), etc.: 100%|##########| 60/60 [00:04<00:00, 13.72it/s]
Animation 59: FadeOut(VGroup of 2 submobjects), etc.: 100%|##########| 60/60 [00:05<00:00, 11.60it/s]
Animation 61: Create(Rectangle), etc.: 100%|##########| 60/60 [00:04<00:00, 13.13it/s]
Animation 62: FadeOut(VGroup of 2 submobjects), etc.: 100%|##########| 60/60 [00:04<00:00, 14.65it/s]
Animation 64: Create(Rectangle), etc.: 100%|#####

Done


CPU times: total: 26min 32s
Wall time: 53min 47s


# End Screen


In [ ]:
name = 'EndScreen'
params = f"-v WARNING -ql --disable_caching --progress_bar leave {name}"
paramsc = f"-v WARNING -ql --progress_bar leave {name}"
paramsk = f"-v WARNING --save_sections -qk --progress_bar leave {name}"
paramssk = f"-v WARNING -s -qk --disable_caching --progress_bar leave {name}"
paramssl = f"-v WARNING -s -ql --progress_bar leave {name}"

class EndScreen(Scene):
    def construct(self):
        
        title = Title('Thank You', match_underline_width_to_text=True, color=BLUE_D)
        title.scale(1.5)
        title.to_edge(UP)

        # References
        text = Text('References', color=WHITE).shift(UP)


        ref1 = '1. McInnes L, Healy J, Astels S. hdbscan: Hierarchical density based clustering. The Journal of Open Source Software. 2017;2(11):205. doi:10.21105/joss.00205'
        ref2 = '2. McInnes L, Healy J. Accelerated Hierarchical Density Based Clustering. Στο: Data Mining Workshops (ICDMW), 2017 IEEE International Conference on. IEEE; 2017:33–42.'
        ref3 = '3. The Manim Community Developers. Manim: Mathematical Animation Framework.; 12 2021. https://www.manim.community/'
        
        ref1 = Text(ref1, font='Montserrat', color=WHITE).scale(0.25).shift(UP)
        ref2 = Text(ref2, font='Montserrat', color=WHITE).scale(0.25)
        ref3 = Text(ref3, font='Montserrat', color=WHITE).scale(0.25).shift(DOWN)

        refs = VGroup(ref1, ref2, ref3).arrange(DOWN, center=False, aligned_edge=LEFT, buff=0.5).next_to(text, DOWN, buff=0.5)

        self.next_section("Thank You", type=PresentationSectionType.SKIP)
        self.play(Write(title))
        
        self.next_section("References", type=PresentationSectionType.NORMAL)
        self.play(Write(text))
        self.wait()
        self.play(FadeIn(refs))
        self.wait(2)

In [ ]:
%%manim $paramsk
plt.rcParams['figure.dpi'] = 300

Animation 3: FadeIn(VGroup of 3 submobjects): 100%|##########| 60/60 [00:16<00:00,  3.63it/s]
